In [1]:
print("Hello World")

Hello World


In [16]:
import random
import pandas as pd
class YahtzeeSection:
    def __init__(self, name, is_set, score, number):
        self.name = name
        self.is_set = is_set
        self.score = score
        self.number = number

class Dice:
    def __init__(self, value):
        self.value = value
        self.locked = False

    def roll_unlocked_dice(self):
        if not self.locked:
            self.value = random.randint(1, 6)
class Game:
    def __init__(self):
        self.dice = [Dice(1), Dice(2), Dice(3), Dice(4), Dice(5)]
        self.sections = [
            YahtzeeSection("Ones", False, 0, 1),
            YahtzeeSection("Twos", False, 0, 2),
            YahtzeeSection("Threes", False, 0, 3),
            YahtzeeSection("Fours", False, 0, 4),
            YahtzeeSection("Fives", False, 0, 5),
            YahtzeeSection("Sixes", False, 0, 6),
        ]
        self.total_rounds = 6
        self.total_score = 0

    def roll_all_dice(self):
        for die in self.dice:
            die.roll_unlocked_dice()

    def lock_die(self, die_number):
        self.dice[die_number].locked = True

    def calculate_score(self):
        for section in self.sections:
            self.total_score += section.score

    def play_round(self):
        for i in range(3):
            self.roll_all_dice()
            print("Roll", i + 1)
            for die in self.dice:
                print(die.value)
            if i < 2:
                response = input("Which dice would you like to lock? (Enter the numbers separated by a space)")
                dice_to_lock = response.replace(',', ' ').split()
                print(dice_to_lock)
                for die in dice_to_lock:
                    self.lock_die(int(die) - 1)
        self.total_score = 0

    def play_game(self):
        for i in range(self.total_rounds):
            self.play_round()
            self.calculate_score()
            print("Total Score:", self.total_score)


    

  



    
        

In [17]:
game = Game()
game.play_game()

Roll 1
6
1
1
1
3
['3', '4', '1']
Roll 2
6
1
1
1
6
[]
Roll 3
6
1
1
1
1
Total Score: 0
Roll 1
6
6
1
1
1
[]
Roll 2
6
6
1
1
5
[]
Roll 3
6
2
1
1
1
Total Score: 0
Roll 1
6
2
1
1
6
[]
Roll 2
6
2
1
1
3
[]
Roll 3
6
5
1
1
4
Total Score: 0
Roll 1
6
3
1
1
1
[]
Roll 2
6
4
1
1
6
[]
Roll 3
6
3
1
1
4
Total Score: 0
Roll 1
6
6
1
1
6
[]
Roll 2
6
4
1
1
6
[]
Roll 3
6
2
1
1
2
Total Score: 0
Roll 1
6
5
1
1
1
[]
Roll 2
6
6
1
1
1
[]
Roll 3
6
1
1
1
6
Total Score: 0


In [ ]:



# Generate five random integers between 1 and 6
random_numbers = [random.randint(1, 6) for _ in range(5)]
print(random_numbers)


[1, 1, 3, 6, 2]
